# FastAPI Example

Example demonstrating how to integrate WorkflowServer into an existing FastAPI application.

This example shows how to:
1. Create a FastAPI application with existing routes
2. Set up WorkflowServer with workflows
3. Mount the workflow server as a sub-application
4. Run the combined application

In [ ]:
%pip install llama-index-workflows-server fastapi

## Define a workflow and setup the server

In [ ]:
%%writefile server.py
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

from workflows import Workflow, step
from workflows.events import StartEvent, StopEvent
from llama_agents.server import WorkflowServer


class UserModel(BaseModel):
    name: str
    email: str

# Existing FastAPI application with some routes
app = FastAPI(title="My API with Workflows", version="1.0.0")


# Existing API routes
@app.get("/")
async def root() -> dict:
    return {"message": "Welcome to My API"}


@app.get("/users/{user_id}")
async def get_user(user_id: int) -> dict:
    return {
        "user_id": user_id,
        "name": f"User {user_id}",
        "email": f"user{user_id}@example.com",
    }


@app.post("/users")
async def create_user(user: UserModel) -> dict:
    return {"message": f"Created user {user.name}", "user": user}


# Define workflows
class UserProcessingWorkflow(Workflow):
    @step
    async def process_user(self, ev: StartEvent) -> StopEvent:
        user_data = getattr(ev, "user_data", {})
        name = user_data.get("name", "Unknown")
        email = user_data.get("email", "unknown@example.com")

        # Simulate some processing
        processed_data = {
            "processed_name": name.upper(),
            "domain": email.split("@")[1] if "@" in email else "unknown",
            "status": "processed",
        }

        return StopEvent(result=processed_data)


class NotificationWorkflow(Workflow):
    @step
    async def send_notification(self, ev: StartEvent) -> StopEvent:
        message = getattr(ev, "message", "Default notification")
        recipient = getattr(ev, "recipient", "admin@example.com")

        # Simulate sending notification
        result = {
            "notification_id": "notif_123",
            "message": message,
            "recipient": recipient,
            "sent_at": "2024-01-01T12:00:00Z",
            "status": "sent",
        }

        return StopEvent(result=result)



def main() -> None:
    # Create workflow server
    workflow_server = WorkflowServer()

    # Register workflows
    workflow_server.add_workflow("user_processing", UserProcessingWorkflow())
    workflow_server.add_workflow("notification", NotificationWorkflow())

    # Mount workflow server as sub-application
    app.mount("/wf-server", workflow_server.app)

    # run the FastAPI server
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")


if __name__ == "__main__":
    main()

## Run the server in background

In [8]:
import subprocess
import time

# Start server in background using subprocess
server_process = subprocess.Popen(["python", "server.py"])
time.sleep(2)  # Wait for server to start
print(f"Server started with PID: {server_process.pid}")

INFO:     Started server process [26386]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Server started with PID: 26386


## Interact with the server

In [9]:
# Confirm existing routes are there
!curl -s http://localhost:8000/users/123

INFO:     127.0.0.1:65035 - "GET /users/123 HTTP/1.1" 200 OK
{"user_id":123,"name":"User 123","email":"user123@example.com"}

In [10]:
# Hit the health endpoint to see the workflow server is available at the path /wf-server
!curl http://localhost:8000/wf-server/health

INFO:     127.0.0.1:65039 - "GET /wf-server/health HTTP/1.1" 200 OK
{"status":"healthy","loaded_workflows":0,"active_workflows":0,"idle_workflows":0}

In [11]:
# List available workflows
!curl http://localhost:8000/wf-server/workflows

INFO:     127.0.0.1:65043 - "GET /wf-server/workflows HTTP/1.1" 200 OK
{"workflows":["user_processing","notification"]}

In [12]:
# Run user processing workflow
!curl -X POST http://localhost:8000/wf-server/workflows/user_processing/run \
  -H "Content-Type: application/json" \
  -d '{"kwargs": {"user_data": {"name": "alice", "email": "alice@company.com"}}}'

INFO:     127.0.0.1:65050 - "POST /wf-server/workflows/user_processing/run HTTP/1.1" 200 OK
{"handler_id":"Mu2qZvQzVu","workflow_name":"user_processing","run_id":"7hMHiP24AL","error":null,"result":{"value":{"result":{"processed_name":"ALICE","domain":"company.com","status":"processed"}},"qualified_name":"workflows.events.StopEvent","type":"StopEvent","types":null},"status":"completed","started_at":"2026-01-29T22:34:33.157083+00:00","updated_at":"2026-01-29T22:34:33.157792+00:00","completed_at":"2026-01-29T22:34:33.157792+00:00"}